# Libraries

In [6]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import re
#pd.set_option('display.max_colwidth', -1)

After web scrapping we created one single data frame with columns 'price', 'locali', 'superficie', 'bagni', 'piano', 'description' and stored it as a .csv file for future usage. 

In [7]:
#Importing data into memory from the stored .csv file
data = pd.read_csv("data_houses.csv", sep='\t', encoding='utf-8')
data.columns

Index(['price', 'locali', 'superficie', 'bagni', 'piano', 'description'], dtype='object')

In [8]:
#The number of rows before cleaning
data.shape

(23203, 6)

In [9]:
#Checking null values of the dataset
data.isnull().sum()

price             1
locali          354
superficie       24
bagni           491
piano          3793
description    5045
dtype: int64

In [10]:
#Checking data types of the dataset
data.dtypes

price          object
locali         object
superficie     object
bagni          object
piano          object
description    object
dtype: object

In [12]:
data.loc[:,['price','locali','superficie','bagni','piano']].describe()

,price,locali,superficie,bagni,piano
count,23202,22849,23179,22712,19410
unique,2335,6,587,315,15
top,€ 199.000,3,90,1,1
freq,338,7177,962,10007,4236


In [5]:
#The values where the number of bathrooms are 1, 2 or 3 should be kept because they are the most common ones and other numbers 
#are rare and therefore considered outliers(they are probably buildings_?)
#also 3+ is eliminated cause it has ambigious meaning

In [11]:
#We decided to drop the rows where the value of the floor is 11+ because there aren't many of those rows and they don't show
data.piano.value_counts()
#eliminate R, A, S and 11+  cause it is ambigious

1      3265
2      2563
T      2285
3      1904
A      1275
4      1216
R       843
5       716
6       374
S       289
7       271
8       104
9        22
11+      19
10        8
Name: piano, dtype: int64

After _analyzing_ the possible values of each attribute of the dataset we decided the following about the process of 
**cleaning** the dataset before making two separate datasets for clustering.
    
The data cleaning process:
    1. Remove nan values
    2. Attribute price should be converted to integer data type and '€' removed from the values
    3. In attribute locali remove rows where the value of locali is 5+ and convert attribute to integer data type
    4. Attribute superficie has some strange values because of the scrapping, like dates '14/02/19' '18/01/19' and all the values that don't make sense should be removed and the attribute should be converted to the integer data type
    5. For attribute piano replace T->0 and drop the rows with piano values A, R, S and 11+ cause they are ambiguous
    
The goal of the cleaning process was to eliminate all the ambiguous values and categorical values that can't have a meaningful numerical representation.

#Clean dataframe

In [13]:
def clean_data(data):
    """
    Method that removes nan values and cleanes the data
    
    Input: dataframe
    Output: cleaned dataframe
    """
    
    #Remove rows where there aren't all values present
    data.dropna(inplace=True)

    #convert to int price and remove € 
    for i in data.index:
    #Becasuse of the web scrapping some prices had strings and text in this waz we dealt with it
        try:
            data.price[i]=int(data.price[i].replace('€', '').strip().replace('.','').replace('da',''))
    #before word class there is price we need
        except:
            try:
                data.price[i]=int(data.price[i].split('class')[0].replace('€', '').strip().replace('.',''))
           #in case of the bad scrapping value (e.g.just text) just drop those rows
            except:
                data.drop(i,inplace=True)
            
    #convert to int superficie
    data.superficie=data.superficie.replace('[/]', '', regex=True).apply(lambda x: int(str(x).replace('.','')))
    #Drop rows where values of locali is 5+ and convert to int
    data['locali']=data['locali'].apply(lambda x: x.strip())
    data.drop(data[ data['locali']=='5+'].index,inplace=True)
#    data.locali=data.locali.apply(lambda x:int(x))


    #Attribute piano replace T->0 drop A, R, S and 11+  cause it is ambigious 
    data.drop(data[(data['piano']=='A') | (data['piano']=='R') | ( data['piano']=='S')| ( data['piano']=='11+')].index,inplace=True)
    #convert to int and piano T is 'terro' which means it is floor 0
    data['piano']=np.where(data['piano']=='T', 0, data['piano'])     
    

#    data.piano=data.piano.apply(lambda x: int(x))
    data.bagni=data.bagni.apply(lambda x: x.strip())
    data.drop(data[( data['bagni']!='1')&( data['bagni']!='2')&( data['bagni']!='3')].index,inplace=True)    
#    data.bagni=data.bagni.apply(lambda x: int(x))
#
    data[["price","locali","piano", "bagni"]] = data[["price","locali","piano", "bagni"]].apply(pd.to_numeric)
    return data

In [14]:
cleaned_data=clean_data(data)

In [16]:
#The number of rows after cleaning the dataset
#10773
cleaned_data.shape

(10773, 6)

In [17]:
#After cleaning the data(dropping rows where there aren't some values) there aren't any NaN values present
cleaned_data.isnull().sum()

price          0
locali         0
superficie     0
bagni          0
piano          0
description    0
dtype: int64

In [18]:
cleaned_data.dtypes

price           int64
locali          int64
superficie      int64
bagni           int64
piano           int64
description    object
dtype: object

# Datasets

Based on the single data frame we extracted using the web scrapping process we created the Description and the Information datasets

## Description dataset

(Description dataset)

    columns: description 

In [25]:
description_dataset=pd.DataFrame(cleaned_data.description)
pd.set_option('display.max_colwidth', -1)
description_dataset.head(1)

,description
1,"papillo eur\r\r\n PAPILLO EUR in elegante complesso residenziale rifinitissimo bilocale composto da soggiorno con angolo cottura, stanza da letto bagno e ampio balcone . con Rifiniture di pregio, pavimenti in parquet / grees, infissi in legno con vetro camera e porte in noce, grate nel salone, riscaldamento termoautonomo con caldaia centralizzata, aria condizionata, videocitofono, porta blindata, serramenti elettrici con chiusura centralizzata, antenna satellitare, isolamento termo acustico, pannelli solari e fotovoltaici , rilevatori elettronici di gas. Tutte le camere sono fornite di impianto antifurto, presa antenna satellitare e presa telefonica.div\r\r\ndiv\r\r\nORARI lunedi chiusidiv\r\r\n martedi 10:00-17:00div\r\r\n mercoledi 10:00-17:00div\r\r\ndivgiovedi 10:00-17:00div\r\r\ndivvenrdi 10:00-17:00div\r\r\ndivsabato 10:00-17:00div\r\r\n domenica 10:00-13:00\r\r\n"


## Information dataset

(Information dataset)

    columns: price, locali, superficie, bagni, piano

In [26]:
information_dataset=cleaned_data.drop(['description'],axis=1)
information_dataset.head(5)

,price,locali,superficie,bagni,piano
1,225000,2,50,1,1
5,339000,3,90,1,4
6,480000,4,125,2,4
7,135000,2,60,1,5
8,249000,2,75,1,1


In [ ]:
#Save information and description datasets to two separate .csv files

In [ ]:
def calculate_tf_idf(df,vocabulary):
    """
    method that computes an inverted index
    
    input:  airbnbdata-just for using the number of files we made
            inverted_idx(dictionary, key=term_id, value=list of document_ids)
            vocabulary(dictionary of all unique words, key=term, value=term_id)
    output: tf_idf_dic(dictionary of tf_idf_values for all docs, key=tuple(term,doc_id ), value=tf_idf value)
    """
    tf_idf_dic=dict()
    #total number of documents
    total_num_docs=df.shape[0]
    result_df=pd.DataFrame()
 #use apply
#for i in df.index:
        #preprocessing 
        df=df.description[i]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        #series of tf values
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        #idf calculation
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs/len(inverted_idx[vocabulary[x]])))
        #combine tf and idf in one result_df dataframe
        result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),pd.Series(idf_calc.values)],axis=1)#.reset_index()
        #multiply tf and idf and create tf_idf column
        result_df['tf_idf']=result_df[1]*result_df[2]
        #key=tuple(term,doc_id), value=tf_idf value
        for idx in range(result_df.shape[0]):
            tf_idf_dic[result_df[0][idx],i]=result_df['tf_idf'][idx]
    return tf_idf_dic

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
every row is a separate house(which is considered as a separate document)

In [ ]:
def build_vocabulary(data):
    """
    Method that creates vocabulary
    
    Input: dataframe in order to access number of files made by that airbnb dataframe
    Output: vocabulary list and doc_vocabs(dictionary, key='doc_id',value=list of unique words belonging to that document)
    """  
    #list for vocabulary 
    vocabulary_lst=[]
    #preprocessing description and title
    df=df.description[0]+' '+df.title[0]
    filtered_words=preprocessing_text(df)
    #temporary variable set used for making vocabulary with unique words
    temp_vocabulary_set=set()
    for word in filtered_words:
        temp_vocabulary_set.add(word)
        vocabulary_lst.append(temp_vocabulary_set)
        doc_vocabs[i]=list(temp_vocabulary_set)
    #union of content of vocabulary_lst
    vocabulary_set=set.union(*vocabulary_lst)
    #mapping words into integers
    vocabulary={} 
    for k,v in enumerate(vocabulary_set):
        vocabulary[v]= k
    return vocabulary


In [ ]:
def preprocessing_text(df):
    """
    Method that returns filtered words from the text input 
    
    Input: string(text)
    Output: list(bag of words)
    """  
    #remove upper cases
    df=df.lower()
   
    #replacing new line sign '\n' '\r' and 'div' from html with a whitespace ' '    
    df=df.replace('\\n',' ').replace('\\r',' ').replace('div',' ')
    
    #for removing stop words
    stop_words = set(stopwords.words('italian')) 
    stop_words.add('div')
    stop_words.add('n')
    stop_words.add('b')

    #remove numbers
    df = re.sub("\d+", "", df)
    
    #for removing punctuations
    tokenizer = RegexpTokenizer(r'\w+')
    
    #to tokenize the string
    word_tokens = tokenizer.tokenize(df)     

    #stemming
    ps = PorterStemmer()
    filtered_words = [ps.stem(w) for w in word_tokens if not w in stop_words] 

    return filtered_words

# Clustering

This step consists in _clustering the house announcements_ using **K-means++** and choosing the **optimal** number of clusters using the **Elbow-Method**.

In [ ]:
#should we normalize?

In [1]:
from wordcloud import WordCloud